In [1]:
import mlflow
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

experiment_name = 'Baseline-async'
mlflow.set_tracking_uri('http://localhost/')

In [2]:
experiment = mlflow.get_experiment_by_name(experiment_name)
df_exp = mlflow.search_runs(experiment_ids=[experiment.experiment_id], filter_string='attributes.status="FINISHED"')

fixed_cols = ['run_id', 'start_time', 'end_time']
for col in df_exp.columns:
    if col.startswith('metrics.') or col.startswith('params'):
        new_col_name = '.'.join(col.split('.')[1:])
        df_exp.rename(columns={col: new_col_name}, inplace=True)
    elif col == 'tags.mlflow.runName':
        df_exp.rename(columns={col: 'run_name'}, inplace=True)
    elif col not in fixed_cols:
        df_exp.drop(columns=[col], inplace=True)
df_exp.head(n=3)

,run_id,start_time,end_time,test_balanced_accuracy,train_f1_weighted,train_time_sec,test_count_observations,train_certainty_neg,test_f1_weighted,train_balanced_accuracy,...,test_predict_time_sec,test_accuracy,train_recall_macro,train_precision_micro,test_precision_weighted,zimp_mechanism,model_type,dataset,random_seed,run_name
0,32c6c37a0bab47a0aa9ac7781b7bf6f1,2022-02-18 22:34:21.414000+00:00,2022-02-18 22:34:34.875000+00:00,0.560406,0.678478,1.303403,500.0,0.489957,0.621646,0.614225,...,0.439959,0.634000,0.614225,0.670213,0.690311,None,DECISION_TREE,TREC-6,36755,DECISION_TREE-105
1,98835f555a34424082c805cabdd44d15,2022-02-18 22:28:22.842000+00:00,2022-02-18 22:34:20.916000+00:00,0.713443,0.719426,119.630484,70000.0,0.471007,0.713335,0.719116,...,21.584111,0.713443,0.719116,0.719116,0.718528,None,DECISION_TREE,DBP-14,45710,DECISION_TREE-110
2,e477a4ba4dac4717b2d51da6beb0bc9f,2022-02-18 22:15:49.078000+00:00,2022-02-18 22:28:22.030000+00:00,0.354600,0.342824,264.185552,50000.0,0.342243,0.340802,0.357672,...,24.278904,0.354600,0.357672,0.357672,0.344660,None,DECISION_TREE,YELP-5,139949,DECISION_TREE-118


In [3]:
df_stats = df_exp.groupby(['dataset', 'model_type']).agg(['mean', 'std', 'count']).fillna(0)
for metric in df_stats.columns.levels[0]:
    df_stats[metric,'ci95'] = 1.96*df_stats[metric]['std']/df_stats[metric]['count']**.5

df_stats = df_stats.drop(['count', 'std'], axis=1, level=1).drop('random_seed', axis=1, level=0).sort_index(axis=1)
df_stats

test_accuracy           test_balanced_accuracy  \
                                     ci95      mean                   ci95   
dataset       model_type                                                     
10K-GNAD      DECISION_TREE      0.000000  0.535019               0.000000   
              FASTTEXT           0.002575  0.268191               0.002360   
              GERMAN_BERT        0.004180  0.717412               0.004732   
              RANDOM_FOREST      0.007675  0.403307               0.006283   
              SVM                0.000000  0.824903               0.000000   
DBP-14        BERT               0.000069  0.988867               0.000069   
              DECISION_TREE      0.000000  0.713443               0.000000   
              FASTTEXT           0.000076  0.983600               0.000076   
              RANDOM_FOREST      0.025222  0.318983               0.025222   
              SVM                0.000000  0.984100               0.000000   
GERMEVAL-2018 DECISION_TREE      0.000000  0.645674               0.000000   
              FASTTEXT           0.000613  0.682225               0.002510   
              GERMAN_BERT        0.002421  0.688670               0.003893   
              RANDOM_FOREST      0.000000  0.661566               0.000000   
              SVM                0.000000  0.681283               0.000000   
GERMEVAL-2020 DECISION_TREE      0.000000  0.486029               0.000000   
              FASTTEXT           0.000591  0.686498               0.000841   
              GERMAN_BERT        0.001694  0.534234               0.002796   
              RANDOM_FOREST      0.000000  0.411005               0.000000   
              SVM                0.000000  0.641196               0.000000   
TREC-6        BERT               0.008102  0.825000               0.023762   
              DECISION_TREE      0.000000  0.634000               0.000000   
              FASTTEXT           0.003670  0.828200               0.003491   
              RANDOM_FOREST      0.032909  0.320200               0.017414   
              SVM                0.000000  0.856000               0.000000   
YELP-5        BERT               0.000325  0.541818               0.000325   
              DECISION_TREE      0.000000  0.354600               0.000000   
              FASTTEXT           0.000238  0.601068               0.000238   
              RANDOM_FOREST      0.005493  0.319502               0.005493   
              SVM                0.000000  0.441320               0.000000   

                                      test_certainty_neg            \
                                 mean               ci95      mean   
dataset       model_type                                             
10K-GNAD      DECISION_TREE  0.504087           0.000023  0.437288   
              FASTTEXT       0.193960           0.001410  0.179484   
              GERMAN_BERT    0.683768           0.002520  0.344503   
              RANDOM_FOREST  0.294119           0.000462  0.169741   
              SVM            0.796318           0.000000  0.645624   
DBP-14        BERT           0.988867           0.001948  0.727565   
              DECISION_TREE  0.713443           0.000000  0.476763   
              FASTTEXT       0.983600           0.000883  0.740631   
              RANDOM_FOREST  0.318983           0.000171  0.072949   
              SVM            0.984100           0.000000  0.632003   
GERMEVAL-2018 DECISION_TREE  0.505191           0.000000  0.684790   
              FASTTEXT       0.555857           0.003263  0.735623   
              GERMAN_BERT    0.561237           0.003251  0.661199   
              RANDOM_FOREST  0.500000           0.000478  0.662618   
              SVM            0.539536           0.000000  0.692076   
GERMEVAL-2020 DECISION_TREE  0.318228           0.000000  0.438769   
              FASTTEXT       0.577714           0.000911  0.637792   
              GERMAN_BERT    0.349641           0.003

In [4]:
for metric in df_stats.columns.levels[0]:
    if metric == 'random_seed':
        continue
    df_stats[metric,'report_str'] = df_stats[metric].apply(lambda row: f"{row['mean']:.3f} \pm {row['ci95']:.2f}", axis=1)

df_report = df_stats.drop(['ci95', 'mean'], axis=1, level=1)
df_report.columns = df_report.columns.droplevel(1)
df_report.head(n=3)

test_accuracy test_balanced_accuracy  \
dataset  model_type                                             
10K-GNAD DECISION_TREE  0.535 \pm 0.00         0.504 \pm 0.00   
         FASTTEXT       0.268 \pm 0.00         0.194 \pm 0.00   
         GERMAN_BERT    0.717 \pm 0.00         0.684 \pm 0.00   

                       test_certainty_neg test_certainty_pos  \
dataset  model_type                                            
10K-GNAD DECISION_TREE     0.437 \pm 0.00     0.673 \pm 0.00   
         FASTTEXT          0.179 \pm 0.00     0.202 \pm 0.00   
         GERMAN_BERT       0.345 \pm 0.00     0.558 \pm 0.00   

                       test_count_observations   test_f1_macro  \
dataset  model_type                                              
10K-GNAD DECISION_TREE       1028.000 \pm 0.00  0.510 \pm 0.00   
         FASTTEXT            1028.000 \pm 0.00  0.117 \pm 0.00   
         GERMAN_BERT         1028.000 \pm 0.00  0.687 \pm 0.00   

                         test_f1_micro test_f1_weighted test_precision_macro  \
dataset  model_type                                                            
10K-GNAD DECISION_TREE  0.535 \pm 0.00   0.527 \pm 0.00       0.537 \pm 0.00   
         FASTTEXT       0.268 \pm 0.00   0.160 \pm 0.00       0.150 \pm 0.01   
         GERMAN_BERT    0.717 \pm 0.00   0.707 \pm 0.00       0.727 \pm 0.01   

                       test_precision_micro  ...  train_f1_micro  \
dataset  model_type                          ...                   
10K-GNAD DECISION_TREE       0.535 \pm 0.00  ...  0.555 \pm 0.00   
         FASTTEXT            0.268 \pm 0.00  ...  0.276 \pm 0.00   
         GERMAN_BERT         0.717 \pm 0.00  ...  0.707 \pm 0.00   

                       train_f1_weighted train_precision_macro  \
dataset  model_type                                              
10K-GNAD DECISION_TREE    0.548 \pm 0.00        0.560 \pm 0.00   
         FASTTEXT         0.168 \pm 0.00        0.167 \pm 0.00   
         GERMAN_BERT      0.702 \pm 0.00        0.724 \pm 0.00   

                       train_precision_micro train_precision_weighted  \
dataset  model_type                                                     
10K-GNAD DECISION_TREE        0.555 \pm 0.00           0.560 \pm 0.00   
         FASTTEXT             0.276 \pm 0.00           0.194 \pm 0.00   
         GERMAN_BERT          0.707 \pm 0.00           0.715 \pm 0.00   

                       train_predict_time_sec train_recall_macro  \
dataset  model_type                                                
10K-GNAD DECISION_TREE        18.263 \pm 0.35     0.511 \pm 0.00   
         FASTTEXT             18.590 \pm 0.66     0.201 \pm 0.00   
         GERMAN_BERT        123.678 \pm 14.31     0.671 \pm 0.00   

                       train_recall_micro train_recall_weighted  \
dataset  model_type                                               
10K-GNAD DECISION_TREE     0.555 \pm 0.00        0.555 \pm 0.00   
         FASTTEXT          0.276 \pm 0.00        0.276 \pm 0.00   
         GERMAN_BERT       0.707 \pm 0.00        0.707 \pm 0.00   

                           train_time_sec  
dataset  model_type                        
10K-GNAD DECISION_TREE   24.270 \pm 23.70  
         FASTTEXT         13.923 \pm 1.60  
         GERMAN_BERT    451.582 \pm 13.20  

[3 rows x 31 columns]

In [5]:
df_dummy = pd.read_csv('dummy_baseline.csv', index_col=0).applymap("{0:.3f}".format)
df_dummy.set_index(pd.MultiIndex.from_tuples([(i, 'DUMMY') for i in df_report.index.levels[0]]), inplace=True)
df_merged = pd.concat([df_report, df_dummy]).sort_index()
df_merged.head(n=3)

test_accuracy test_balanced_accuracy  \
dataset  model_type                                             
10K-GNAD DECISION_TREE  0.535 \pm 0.00         0.504 \pm 0.00   
         DUMMY                   0.163                  0.111   
         FASTTEXT       0.268 \pm 0.00         0.194 \pm 0.00   

                       test_certainty_neg test_certainty_pos  \
dataset  model_type                                            
10K-GNAD DECISION_TREE     0.437 \pm 0.00     0.673 \pm 0.00   
         DUMMY                      0.163              0.163   
         FASTTEXT          0.179 \pm 0.00     0.202 \pm 0.00   

                       test_count_observations   test_f1_macro  \
dataset  model_type                                              
10K-GNAD DECISION_TREE       1028.000 \pm 0.00  0.510 \pm 0.00   
         DUMMY                        1028.000           0.031   
         FASTTEXT            1028.000 \pm 0.00  0.117 \pm 0.00   

                         test_f1_micro test_f1_weighted test_precision_macro  \
dataset  model_type                                                            
10K-GNAD DECISION_TREE  0.535 \pm 0.00   0.527 \pm 0.00       0.537 \pm 0.00   
         DUMMY                   0.163            0.046                0.018   
         FASTTEXT       0.268 \pm 0.00   0.160 \pm 0.00       0.150 \pm 0.01   

                       test_precision_micro  ...  train_f1_micro  \
dataset  model_type                          ...                   
10K-GNAD DECISION_TREE       0.535 \pm 0.00  ...  0.555 \pm 0.00   
         DUMMY                        0.163  ...           0.163   
         FASTTEXT            0.268 \pm 0.00  ...  0.276 \pm 0.00   

                       train_f1_weighted train_precision_macro  \
dataset  model_type                                              
10K-GNAD DECISION_TREE    0.548 \pm 0.00        0.560 \pm 0.00   
         DUMMY                     0.046                 0.018   
         FASTTEXT         0.168 \pm 0.00        0.167 \pm 0.00   

                       train_precision_micro train_precision_weighted  \
dataset  model_type                                                     
10K-GNAD DECISION_TREE        0.555 \pm 0.00           0.560 \pm 0.00   
         DUMMY                         0.163                    0.027   
         FASTTEXT             0.276 \pm 0.00           0.194 \pm 0.00   

                       train_predict_time_sec train_recall_macro  \
dataset  model_type                                                
10K-GNAD DECISION_TREE        18.263 \pm 0.35     0.511 \pm 0.00   
         DUMMY                            NaN              0.111   
         FASTTEXT             18.590 \pm 0.66     0.201 \pm 0.00   

                       train_recall_micro train_recall_weighted  \
dataset  model_type                                               
10K-GNAD DECISION_TREE     0.555 \pm 0.00        0.555 \pm 0.00   
         DUMMY                      0.163                 0.163   
         FASTTEXT          0.276 \pm 0.00        0.276 \pm 0.00   

                          train_time_sec  
dataset  model_type                       
10K-GNAD DECISION_TREE  24.270 \pm 23.70  
         DUMMY                       NaN  
         FASTTEXT        13.923 \pm 1.60  

[3 rows x 31 columns]

In [6]:
def get_latex_method_name(model_type):
    if model_type == 'GERMAN_BERT':
        return 'BERT'
    elif model_type == 'DECISION_TREE':
        return 'DT'
    elif model_type == 'RANDOM_FOREST':
        return 'RF'
    elif model_type == 'FASTTEXT':
        return 'FT'
    elif model_type == 'DUMMY':
        return 'BASE'
    return model_type.replace('_', '\_')

def get_latex_metric(metric_str, max_score, method):
    is_max = metric_str == max_score
    
    if method == 'SVM':
        metric_str = metric_str.split()[0]
    
    if metric_str.endswith('0.00'):
        metric_str = metric_str[:-4] + '\epsilon'
    if is_max:
        return '\mathbf{' + metric_str + '}'
    return metric_str

In [7]:
reported_metrics = ['train_accuracy', 'train_balanced_accuracy', 'train_f1_weighted', 'test_accuracy', 'test_balanced_accuracy', 'test_f1_weighted']
reported_methods = ['DUMMY', 'DECISION_TREE', 'RANDOM_FOREST', 'SVM', 'FASTTEXT', 'BERT', 'GERMAN_BERT']

print("""

""")

for dataset in df_merged.index.levels[0]:
    df_dataset = df_merged.loc[dataset]
    max_scores = df_dataset.fillna('0').max()
    print("""
\\begin{table}[]
    \small
    \centering
    \\begin{tabular}{l|ccc|ccc}
        Method & $ACC_{train}$ & $BAC_{train}$ & $F_{train}$ & $ACC_{test}$ & $BAC_{test}$ & $F_{test}$ \\\\ \hline
    """)

    for method in reported_methods:
        if method not in df_dataset.index:
            continue
        s = ' ' * 4
        s += '\\textit{' + get_latex_method_name(method) + '} & '
        s += ' & '.join([f'${get_latex_metric(df_dataset.loc[method, metric], max_scores[metric], method)}$' for metric in reported_metrics])
        s +=  ' \\\\'
        print(s)
    
    print("""
    \end{tabular}
    \caption{Baseline results for \\texttt{""" + dataset + """}}
    \label{tab:results_"""+dataset+"""}
\end{table}
    """)





\begin{table}[]
    \small
    \centering
    \begin{tabular}{l|ccc|ccc}
        Method & $ACC_{train}$ & $BAC_{train}$ & $F_{train}$ & $ACC_{test}$ & $BAC_{test}$ & $F_{test}$ \\ \hline
    
    \textit{BASE} & $0.163$ & $0.111$ & $0.046$ & $0.163$ & $0.111$ & $0.046$ \\
    \textit{DT} & $0.555 \pm \epsilon$ & $0.511 \pm \epsilon$ & $0.548 \pm \epsilon$ & $0.535 \pm \epsilon$ & $0.504 \pm \epsilon$ & $0.527 \pm \epsilon$ \\
    \textit{RF} & $0.422 \pm 0.01$ & $0.308 \pm \epsilon$ & $0.338 \pm 0.01$ & $0.403 \pm 0.01$ & $0.294 \pm 0.01$ & $0.319 \pm 0.01$ \\
    \textit{SVM} & $\mathbf{0.996}$ & $\mathbf{0.997}$ & $\mathbf{0.996}$ & $\mathbf{0.825}$ & $\mathbf{0.796}$ & $\mathbf{0.825}$ \\
    \textit{FT} & $0.276 \pm \epsilon$ & $0.201 \pm \epsilon$ & $0.168 \pm \epsilon$ & $0.268 \pm \epsilon$ & $0.194 \pm \epsilon$ & $0.160 \pm \epsilon$ \\
    \textit{BERT} & $0.707 \pm \epsilon$ & $0.671 \pm \epsilon$ & $0.702 \pm \epsilon$ & $0.717 \pm \epsilon$ & $0.684 \pm \epsilon$ & $0.